<a href="https://colab.research.google.com/github/AiProcess/torch_deep_rl/blob/main/04_getting_started_logging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Get started with logging

**Author**: [Vincent Moens](https://github.com/vmoens)


<div class="alert alert-info"><h4>Note</h4><p>To run this tutorial in a notebook, add an installation cell
  at the beginning containing:

```
!pip install tensordict
!pip install torchrl</p></div>
```


In [1]:
!pip install tensordict
!pip install torchrl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.w

The final chapter of this series before we orchestrate everything in a
training script is to learn about logging.

## Loggers

Logging is crucial for reporting your results to the outside world and for
you to check that your algorithm is learning properly. TorchRL has several
loggers that interface with custom backends such as
wandb (:class:`~torchrl.record.loggers.wandb.WandbLogger`),
tensorboard (:class:`~torchrl.record.loggers.tensorboard.TensorBoardLogger`) or a lightweight and
portable CSV logger (:class:`~torchrl.record.loggers.csv.CSVLogger`) that you can use
pretty much everywhere.

Loggers are located in the ``torchrl.record`` module and the various classes
can be found in the `API reference <ref_loggers>`.

We tried to keep the loggers APIs as similar as we could, given the
differences in the underlying backends. While execution of the loggers will
mostly be interchangeable, their instantiation can differ.

Usually, building a logger requires
at least an experiment name and possibly a logging directory and other
hyperparameters.




In [2]:
from torchrl.record import CSVLogger

logger = CSVLogger(exp_name="my_exp")

Once the logger is instantiated, the only thing left to do is call the
logging methods! For example, :meth:`~torchrl.record.CSVLogger.log_scalar`
is used in several places across the training examples to log values such as
reward, loss value or time elapsed for executing a piece of code.



In [ ]:
logger.log_scalar("my_scalar", 0.4)

## Recording videos

Finally, it can come in handy to record videos of a simulator. Some
environments (e.g., Atari games) are already rendered as images whereas
others require you to create them as such. Fortunately, in most common cases,
rendering and recording videos isn't too difficult.

Let's first see how we can create a Gym environment that outputs images
alongside its observations. :class:`~torchrl.envs.GymEnv` accept two keywords
for this purpose:
- ``from_pixels=True`` will make the env ``step`` function
write a ``"pixels"`` entry containing the images corresponding to your
observations, and

- ``pixels_only=False`` will indicate that you want the
observations to be returned as well.




In [ ]:
from torchrl.envs import GymEnv

env = GymEnv("CartPole-v1", from_pixels=True, pixels_only=False)

print(env.rollout(max_steps=3))

from torchrl.envs import TransformedEnv

We now have built an environment that renders images with its observations.
To record videos, we will need to combine that environment with a recorder
and the logger (the logger providing the backend to save the video).
This will happen within a transformed environment, like the one we saw in
the `first tutorial <gs_env_ted>`.



In [ ]:
from torchrl.record import VideoRecorder

recorder = VideoRecorder(logger, tag="my_video")
record_env = TransformedEnv(env, recorder)

When running this environment, all the ``"pixels"`` entries will be saved in
a local buffer (i.e. RAM) and dumped in a video on demand (to prevent excessive
RAM usage, you are advised to call this method whenever appropriate!):



In [ ]:
rollout = record_env.rollout(max_steps=3)
# Uncomment this line to save the video on disk:
# recorder.dump()

In this specific case, the video format can be chosen when instantiating
the CSVLogger.

(If you want to customise how your video is recorded, have a look at `our knowledge base <ref_knowledge_base>`.)

This is all we wanted to cover in the getting started tutorial.
You should now be ready to code your
`first training loop with TorchRL <gs_first_training>`!


